## Introduction

This data comes from FiveThirtyEight using their RAPTOR Metric. No, this isn't just for the Toronto Raptors. It's a metric they created to better reflects the style of the modern NBA and the performance of its players. It standard for Robust Algorithm (using) Player Tracking (and) On/Off Ratings. It also has WAR (Wins Above Replacement Data). That's more up my ally as a baseball fan.
\
\
Full Info:
\
https://fivethirtyeight.com/features/introducing-raptor-our-new-metric-for-the-modern-nba/

## Questions
Who is the best player since 1974 (when the data starts)?
What is the best single season of all time (defensive, offensive, total)
What young player is on the path to be one of the all-time greats?